In [17]:
from ContentBasedFiltering import ContentBasedFiltering
from sklearn.metrics import root_mean_squared_error, mean_squared_error, mean_absolute_error, f1_score
from scipy import stats

from surprise import Dataset, Reader
from surprise import KNNBasic, KNNWithMeans
from surprise.model_selection import train_test_split

import pandas as pd
import numpy as np
import utils_local

## Evalaution functions

In [18]:
def f(df_species: pd.DataFrame):
    if np.sum(df_species["occurence"]) == 0:
        return 0

    preferences = df_species["similarity"]
    percentile_rank = 1 - stats.percentileofscore(preferences, preferences) / 100

    expected_rank_species = np.sum(df_species["occurence"] * percentile_rank) / np.sum(df_species["occurence"])

    return expected_rank_species

In [19]:
def calc_expected_percentile_rank(df_pred: pd.DataFrame) -> float:
    """Calculate the expected percentile rank as in paper "Collaborative Filtering for Implicit Feedback Datasets"

    Args:
        df_pred (pd.DataFrame): prediction dataframe

    Returns:
        float: expected percentile rank
    """
    expected_ranks = (
        df_pred.sort_values(by="similarity")
        .groupby(by="genus")
        .apply(f, include_groups=False)
    )
    expected_percentile_rank = expected_ranks[expected_ranks > 0].mean()

    return expected_percentile_rank

## Preprocessing train and test data

In [20]:
# Reading train and test data
PATH_RAW_DATA = "../data/"
df_raw_data = pd.read_csv(PATH_RAW_DATA + "AllSites_SiteOccurrences_AllGenera_26.1.24.csv")
df_genus_data = pd.read_csv(PATH_RAW_DATA + "FossilGenera_MammalMassDiet_Jan24.csv", sep=",")
df_dental_data = pd.read_csv(PATH_RAW_DATA + "DentalTraits_Genus_PPPA_ds.csv", sep=",")

display(df_raw_data)
display(df_genus_data)
display(df_dental_data)

,SITE_NAME,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,...,Total_Gen_Count,Large_GenCount,Small_GenCount,smallperlarge,smallprop,Herb_GenCount,Nonherb_GenCount,DietRatio,HerbProp,mid_age
0,Aba Zawei,1,1,1,1,0,0,0,0,0,...,4,4,0,0.00,0.000000,4,0,NaN,1.000000,0.0265
1,Abric Romani,1,0,1,0,1,1,1,1,1,...,12,12,0,0.00,0.000000,6,5,1.2,0.500000,0.0550
2,Acheng_Jiaojie,0,0,0,0,0,0,1,0,0,...,7,5,2,0.40,0.285714,5,2,2.5,0.714286,0.2100
3,Adler cave,1,0,0,0,0,1,0,1,0,...,10,5,5,1.00,0.500000,6,4,1.5,0.600000,0.0275
4,Adyrgan,1,0,0,1,0,0,0,0,0,...,11,5,6,1.20,0.545455,11,0,NaN,1.000000,2.2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,Zhoukoudian_Upper Cave_sapiens,1,0,1,1,1,1,1,1,1,...,39,25,14,0.56,0.358974,19,19,1.0,0.487179,0.0343
714,Ziyang_B site,1,0,0,0,0,0,1,0,0,...,8,8,0,0.00,0.000000,7,0,NaN,0.875000,0.0350
715,Zuurland,0,0,0,0,0,0,0,0,0,...,7,0,7,NaN,1.000000,0,7,0.0,0.000000,2.2000
716,Zuurland (-42 to -46 m),0,0,0,0,0,0,0,0,0,...,5,0,5,NaN,1.000000,0,5,0.0,0.000000,1.5500


,Genus,Order,Family,MassSource,Massg,LogMass,LargeSmall,SizeClass,Diet,DietSource
0,Abudhabia,Rodentia,Muridae,Family average,1.343147e+02,2.128124,Small,small,Herbivore,Phylacine
1,Aceratherium,Perissodactyla,Rhinocerotidae,Cooke,1.099006e+06,6.041000,Large,large,Herbivore,Phylacine
2,Acinonyx,Carnivora,Felidae,Phylacine,4.670000e+04,4.669317,Large,large,Non-Herbivore,Phylacine
3,Aepyosciurus,Rodentia,Sciuridae,NOW,2.860000e+02,2.456366,Small,small,Herbivore,Phylacine
4,Aeretes,Rodentia,Sciuridae,Phylacine,7.324000e+02,2.864748,Small,small,Herbivore,Phylacine
...,...,...,...,...,...,...,...,...,...,...
569,Xenocyon,Carnivora,Canidae,Cooke,4.759825e+04,4.677591,Large,large,Non-Herbivore,GroupAv
570,Yangia,Rodentia,Muridae,Family average,1.343147e+02,2.128124,Small,small,NaN,NaN
571,Yanshuella,Eulipotyphla,Talpidae,Family average,7.842381e+01,1.894448,Small,small,Non-Herbivore,GroupAv
572,Zelceina,Eulipotyphla,Soricidae,Family average,1.145527e+01,1.059005,Small,small,Non-Herbivore,comparable to Sorex


,Genus,n,massg,HY,LOP,AL,OL,SF,BUN,OT,Excl_AL,ConsInGenhyp
0,Addax,1.0,7.000030e+04,3,2,0,1,1.0,0,0.0,0,True
1,Aepyceros,1.0,5.250010e+04,3,2,0,1,0.0,0,0.0,0,True
2,Alcelaphus,1.0,1.710015e+05,3,2,0,1,0.0,0,1.0,0,True
3,Alces,1.0,3.569980e+05,1,2,1,0,0.0,0,0.0,1,True
4,Allochrocebus,3.0,5.708333e+03,1,0,0,0,0.0,1,0.0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
197,Praemegaceros,NaN,7.000000e+04,1,2,1,0,0.0,0,0.0,1,True
198,Sinomegaceros,NaN,3.638878e+05,1,2,1,0,0.0,0,0.0,1,True
199,Soergelia,NaN,2.250000e+05,2,2,0,1,0.0,0,0.0,0,True
200,Spirocerus,NaN,7.906786e+05,2,2,1,1,0.0,0,0.0,0,True


In [21]:
dental_cols = [
    "Genus",
    "HY",
    "LOP",
    "AL",
    "OL",
    "SF",
    "BUN",
    "OT",
    "Excl_AL"
]

df_dental_data = df_dental_data[dental_cols]
display(df_dental_data)

,Genus,HY,LOP,AL,OL,SF,BUN,OT,Excl_AL
0,Addax,3,2,0,1,1.0,0,0.0,0
1,Aepyceros,3,2,0,1,0.0,0,0.0,0
2,Alcelaphus,3,2,0,1,0.0,0,1.0,0
3,Alces,1,2,1,0,0.0,0,0.0,1
4,Allochrocebus,1,0,0,0,0.0,1,0.0,0
...,...,...,...,...,...,...,...,...,...
197,Praemegaceros,1,2,1,0,0.0,0,0.0,1
198,Sinomegaceros,1,2,1,0,0.0,0,0.0,1
199,Soergelia,2,2,0,1,0.0,0,0.0,0
200,Spirocerus,2,2,1,1,0.0,0,0.0,0


In [22]:
# With genus info, give the columns you want to use and convert categorical using one-hot-encoding
genus_info_cols = [
    "Genus",
    "Order",
    "Family",
    "Massg",
    "Diet",
    "DietSource"
]
        
df_genus_data = df_genus_data[genus_info_cols]

dummy_cols = [
    "Order",
    "Family",
    "Diet",
    "DietSource"
]

#The genus column must be the first one in genus data
df_genus_data = pd.get_dummies(df_genus_data, columns=dummy_cols)
df_genus_data = df_genus_data.replace({False: 0, True: 1})
df_genus_data = df_genus_data.merge(df_dental_data, "left", on="Genus")


PATH_DIR_DATA_PROCESS = "../data/data_processed/"

data_train = np.load(PATH_DIR_DATA_PROCESS + "data_train.npy", allow_pickle=True)
data_val = np.load(PATH_DIR_DATA_PROCESS + "data_val.npy", allow_pickle=True)

df_train = utils_local.conv_dataset_patch2df(data_train)
df_val = utils_local.conv_dataset_patch2df(data_val)

print(f"train: {df_train.shape}")
print(f"val: {df_val.shape}")

display(df_genus_data.head())
display(df_train.head())
display(df_val.head())

/var/folders/5m/7ljzc1r113358z1s1_rs2k4r0000gn/T/ipykernel_8323/1439918365.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_genus_data = df_genus_data.replace({False: 0, True: 1})


train: (304000, 3)
val: (20536, 3)


,Genus,Massg,Order_Artiodactyla,Order_Carnivora,Order_Cetartiodactyla,Order_Chiroptera,Order_Dermoptera,Order_Eulipotyphla,Order_Hyracoidea,Order_Lagomorpha,...,DietSource_comparable to Sorex,DietSource_comparable(mis-spelling?) to Rhizomyiddes,HY,LOP,AL,OL,SF,BUN,OT,Excl_AL
0,Abudhabia,1.343147e+02,0,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aceratherium,1.099006e+06,0,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acinonyx,4.670000e+04,0,1,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aepyosciurus,2.860000e+02,0,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aeretes,7.324000e+02,0,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,site,species,occurence
0,564,348,0.0
1,564,349,0.0
2,564,350,0.0
3,564,351,0.0
4,565,348,0.0


,site,species,occurence
0,198,116,0.0
1,198,117,0.0
2,198,118,0.0
3,198,119,0.0
4,199,116,0.0


In [23]:
# Deal with the missing values in df_genus_data
df_genus_data = df_genus_data.fillna(-1)

In [24]:
# Encrypting the genus and site information
path_dir_encode = PATH_DIR_DATA_PROCESS

path_enc_genera = path_dir_encode + "ordinal_enc_species.json"
path_enc_site = path_dir_encode + "ordinal_enc_site.json"

enc_genera = utils_local.CategoryDict.from_file(path_enc_genera)
enc_site = utils_local.CategoryDict.from_file(path_enc_site)

In [25]:
df_train["site"] = df_train["site"].map(enc_site.dict_id2name)
df_train["species"] = df_train["species"].map(enc_genera.dict_id2name)

df_val["site"] = df_val["site"].map(enc_site.dict_id2name)
df_val["species"] = df_val["species"].map(enc_genera.dict_id2name)

# Renaming columns
df_train = df_train.rename(columns={"site": "SITE_NAME", "species": "genus"})
df_val = df_val.rename(columns={"site": "SITE_NAME", "species": "genus"})

display(df_train.head())
display(df_val.head())

,SITE_NAME,genus,occurence
0,Tam Hang,Vespertilio,0.0
1,Tam Hang,Papio,0.0
2,Tam Hang,Cynocephalus,0.0
3,Tam Hang,Melursus,0.0
4,Tam Nang,Vespertilio,0.0


,SITE_NAME,genus,occurence
0,Grays Thurrock,Elephas,0.0
1,Grays Thurrock,Hystrix,0.0
2,Grays Thurrock,Cuon,0.0
3,Grays Thurrock,Rhinolophus,0.0
4,Grosse Grotte (Blaubeuren),Elephas,0.0


In [26]:
# Reshaping into matrix form for the algorithm
df_train_non_matrix = df_train.copy()
df_train = pd.pivot(df_train, index="SITE_NAME", columns="genus", values="occurence").fillna(0)
display(df_train.head())

genus,Acinonyx,Aepyosciurus,Aeretes,Ailuropoda,Ailurus,Alactagulus,Alcelaphus,Alces,Algarolutra,Alilepus,...,Villanyia,Viverra,Viverravus,Viverricula,Vormela,Vulpes,Wushanomys,Xenocyon,Yangia,Zygolophodon
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abric Romani,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Acheng_Jiaojie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adler cave,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Adyrgan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# The site information must be included into matrix for the algorithm
cols_redundant = ["SITE_NAME",
    'LAT',
    'LONG',
    'MAX_AGE',
    'MIN_AGE',
    'age_range',
    'Large_GenCount',
    'Small_GenCount',
    'Herb_GenCount',
    'Nonherb_GenCount',
    'mid_age'
    ]

df_redundant = df_raw_data[cols_redundant]
df_train = df_train.merge(df_redundant, how="left", left_index=True, right_on="SITE_NAME")

desired_column_order = ['SITE_NAME'] + [col for col in df_train.columns if col != 'SITE_NAME']
df_train = df_train[desired_column_order]

df_train.head()

,SITE_NAME,Acinonyx,Aepyosciurus,Aeretes,Ailuropoda,Ailurus,Alactagulus,Alcelaphus,Alces,Algarolutra,...,LAT,LONG,MAX_AGE,MIN_AGE,age_range,Large_GenCount,Small_GenCount,Herb_GenCount,Nonherb_GenCount,mid_age
0,Aba Zawei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,33.250000,102.416667,0.0295,0.0235,0.006,4,0,4,0,0.0265
1,Abric Romani,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.530754,1.679613,0.0700,0.0400,0.030,12,0,6,5,0.0550
2,Acheng_Jiaojie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45.351944,127.088056,0.2660,0.1540,0.112,5,2,5,2,0.2100
3,Adler cave,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.250000,16.667000,0.0450,0.0100,0.035,5,5,6,4,0.0275
4,Adyrgan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,43.050000,80.200000,2.5000,1.9000,0.600,5,6,11,0,2.2000


## Training the model and making predictions

In [28]:
# Train the algorithm with train dataset
cbf = ContentBasedFiltering()
cbf.fit(df_train, df_genus_data, n_site_columns=10, normalization="min-max")

WARNING! Genus info data contains nans. Assigning to zeros
WARNING! Site info data contains nans. Assigning to zeros


In [29]:
# Predictions and true values
true_and_pred = cbf.predict(df_val)
true_and_pred = true_and_pred.fillna(0)
true_and_pred.head()

,SITE_NAME,genus,occurence,similarity
15883,Aba Zawei,Bison,0.0,0.851957
19801,Aba Zawei,Procapra,0.0,0.845164
6473,Aba Zawei,Ammotragus,0.0,0.788996
15403,Aba Zawei,Hippopotamus,0.0,0.718846
15882,Aba Zawei,Mammuthus,0.0,0.647499


## Evaluating

In [30]:
epr = calc_expected_percentile_rank(true_and_pred)
rms = root_mean_squared_error(true_and_pred["occurence"], true_and_pred["similarity"])
mae = mean_absolute_error(true_and_pred["occurence"], true_and_pred["similarity"])

print("Expected percentile rank:", epr, "RMS:", rms, "MAE:", mae)

Expected percentile rank: 0.4011551521862664 RMS: 0.4051199321996161 MAE: 0.3699152170316053


## Checking the site info and genus info matrices

In [31]:
site_info = cbf.site_info_with_genus_info
genus_info = cbf.genus_info_with_site_info

# Evaluating collaborative-knn

In [53]:
df_raw_train = df_raw_data.iloc[:, :-21].set_index("SITE_NAME")
df_raw_train_non_matrix = df_raw_train.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})
df_raw_train_non_matrix

,SITE_NAME,genus,presence
0,Aba Zawei,Equus,1
1,Aba Zawei,Coelodonta,1
2,Aba Zawei,Bos,1
3,Aba Zawei,Gazella,1
4,Aba Zawei,Ursus,0
...,...,...,...
324531,Zverinogolovskoe,Sinoryx,0
324532,Zverinogolovskoe,Prospalax,0
324533,Zverinogolovskoe,Pliopetaurista,0
324534,Zverinogolovskoe,Predicrostonyx,0


In [69]:
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df_train_non_matrix, reader) # Column order must be user, item, rating

sim_options = {
    'name': "MSD",
    'user_based': True  # True for user-user, False for item-item
}

trainset = data.build_full_trainset()

knn = KNNBasic(k=5, min_k=1, sim_options=sim_options)
knn.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [70]:
data_test = Dataset.load_from_df(df_val, reader)
data_test = data_test.build_full_trainset()
testset = data_test.build_testset()

# Get predictions for all user-item pairs
predictions = knn.test(testset)

# Get item scores from the predictions
item_scores = [(prediction.uid, prediction.iid, prediction.est) for prediction in predictions]
knn_pred = pd.DataFrame(item_scores, columns =['SITE_NAME', 'PREDICTED_GENUS', 'similarity'])

knn_pred

,SITE_NAME,PREDICTED_GENUS,similarity
0,Grays Thurrock,Elephas,0.0
1,Grays Thurrock,Hystrix,0.0
2,Grays Thurrock,Cuon,0.0
3,Grays Thurrock,Rhinolophus,0.0
4,Grays Thurrock,Nyctereutes,0.0
...,...,...,...
20531,Cherevichnoe 1,Melursus,0.0
20532,Cherevichnoe 1,Lycaon,0.0
20533,Cherevichnoe 1,Tetraceros,0.0
20534,Cherevichnoe 1,Boselaphus,0.0


In [71]:
knn_pred = knn_pred.merge(df_val, how="left", left_on=["SITE_NAME", "PREDICTED_GENUS"], right_on=["SITE_NAME", "genus"])
knn_pred

,SITE_NAME,PREDICTED_GENUS,similarity,genus,occurence
0,Grays Thurrock,Elephas,0.0,Elephas,0.0
1,Grays Thurrock,Hystrix,0.0,Hystrix,0.0
2,Grays Thurrock,Cuon,0.0,Cuon,0.0
3,Grays Thurrock,Rhinolophus,0.0,Rhinolophus,0.0
4,Grays Thurrock,Nyctereutes,0.0,Nyctereutes,0.0
...,...,...,...,...,...
20531,Cherevichnoe 1,Melursus,0.0,Melursus,0.0
20532,Cherevichnoe 1,Lycaon,0.0,Lycaon,0.0
20533,Cherevichnoe 1,Tetraceros,0.0,Tetraceros,0.0
20534,Cherevichnoe 1,Boselaphus,0.0,Boselaphus,0.0


In [72]:
epr = calc_expected_percentile_rank(knn_pred)
rms = root_mean_squared_error(knn_pred["occurence"], knn_pred["similarity"])
mae = mean_absolute_error(knn_pred["occurence"], knn_pred["similarity"])

print("Expected percentile rank:", epr, "RMS:", rms, "MAE:", mae)

Expected percentile rank: 0.2764608473912176 RMS: 0.13925721777407116 MAE: 0.030384262927097862
